In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from datetime import datetime
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('future.no_silent_downcasting', True)
from bkanalysis.managers import DataManager, MarketManager, TransformationManager, FigureManager
from bkanalysis.salary import Salary, create_default

ref_currency = 'USD'

In [ ]:
data_manager = DataManager()
data_manager.load_pregenerated_data('data_manager.csv')

In [ ]:
market_manager = MarketManager(ref_currency)
market_manager.load_pregenerated_data('data_market.csv')

In [ ]:
transformation_manager = TransformationManager(data_manager, market_manager)
transformation_manager.group_transaction()
figure_manager = FigureManager(transformation_manager)

date_range = [datetime(2025,1,1), datetime(2025,12,31)]
account = None

In [ ]:
figure_manager.get_figure_timeseries()

In [ ]:
figure_manager.get_figure_sunburst(date_range=date_range, account=account, how="both")

In [ ]:
category = {'FullSubType': 'Grocery'}
label = 'MemoMapped'
show_count = 6
figure_manager.get_figure_bar(category, label, show_count, date_range, account)

In [ ]:
category = {'FullSubType': 'Grocery'}
label = 'MemoMapped'
show_count = 6
figure_manager.get_figure_bar(category, label, show_count, date_range, account, return_bar=False)

In [ ]:
figure_manager.get_category_breakdown(category, label, show_count, date_range, account)

In [ ]:
figure_manager.get_figure_waterfall(date_range, account)

In [ ]:
figure_manager.get_figure_waterfall(date_range, account, salary_override=None, include_capital_gain=False)

In [ ]:
figure_manager.get_figure_waterfall(date_range, account, salary_override=create_default(transformation_manager), include_capital_gain=False)

In [ ]:
data, fig = figure_manager.get_capital_gain_brkdn(date_range, 0.99)
fig

In [ ]:
transactions = transformation_manager.data_manager.transactions.copy(deep=True)
transactions = transactions[['Date', 'Quantity', 'FullType', 'FullSubType', 'FullMasterType']]
transactions = transactions[transactions.Date > datetime(2024, 1, 1)]
transactions = transactions[~transactions.FullMasterType.isin(['Intra-Account Transfers', "Tax"])]
transactions["Month"] = transactions["Date"].dt.to_period("M")
trans_pivot = pd.pivot_table(transactions, index=['Month'], columns="FullMasterType", values='Quantity', aggfunc='sum')
trans_pivot.plot.bar(grid=True, stacked=True, figsize=(12,6))

In [ ]:
transactions = transformation_manager.data_manager.transactions.copy(deep=True)
transactions = transactions[['Date', 'Quantity', 'FullType', 'FullSubType', 'FullMasterType']]
transactions = transactions[transactions.Date > datetime(2024, 1, 1)]
transactions = transactions[transactions.FullMasterType.isin(['Income'])]
transactions = transactions[transactions.FullType.isin(['Salary'])]
transactions["Month"] = transactions["Date"].dt.to_period("M")
trans_pivot = pd.pivot_table(transactions, index=['Month'], columns="FullSubType", values='Quantity', aggfunc='sum')
trans_pivot.plot.bar(grid=True, stacked=True, figsize=(12,6))

In [ ]:
def get_saving_ratio(year=2025, month=None, include_capital=False, include_bonus=False):
    transactions = transformation_manager.data_manager.transactions.copy(deep=True)
    transactions = transactions[['Date', 'Quantity', 'FullType', 'FullSubType', 'FullMasterType']]
    transactions = transactions[transactions.Date.dt.year == year]
    if month is not None:
        transactions = transactions[transactions.Date.dt.month == month]
    df_income = transactions[transactions.FullMasterType.isin(['Income'])]
    transactions = transactions[~transactions.FullMasterType.isin(['Income', 'Intra-Account Transfers', 'Ex. Income'])]

    expenses = transactions.Quantity.sum()
    
    if not include_capital:
        df_income = df_income[(df_income.FullType == 'Salary')]
    if not include_bonus:
        df_income = df_income[(df_income.FullSubType != 'UBS Bonus')]

    income = df_income.Quantity.sum()
    return (income + expenses) / income


In [ ]:
yearly_saving_ratios = {}
for y in range(2014, 2026):
    yearly_saving_ratios[y] = get_saving_ratio(y)

In [ ]:
monthly_saving_ratios = {}
for m in range(1, 6):
    monthly_saving_ratios[m] = get_saving_ratio(2025, m)

In [ ]:
monthly_saving_ratios

In [ ]:
pd.Series(yearly_saving_ratios).plot.bar(grid=True, figsize=(10,5), title='Yearly Saving Ratio')

In [ ]:
pd.Series(monthly_saving_ratios).plot.bar(grid=True, figsize=(10,5), title='Monthly Saving Ratio')

In [ ]:
figure_manager.get_saving_rate_gauge(28, 25)